<a href="https://colab.research.google.com/github/basharE/CVDs_Predictors/blob/main/CVDs_Predictors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages/Imports

In [38]:
!pip install -U -q PyDrive
# !pip install shap
# !pip install boruta
# !pip install BorutaShap
# !pip install catboost
# !pip install verstack

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2
from scipy.special import softmax
# from boruta import BorutaPy
# from BorutaShap import BorutaShap
# from verstack import LGBMTuner
from sklearn.model_selection import cross_val_score

import pandas as pd
import numpy as np
import seaborn as sn
sn.set_style(style='darkgrid')
import matplotlib.pyplot as plt
import math

# Uploading data 

## Connecting to google drive

In [39]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Uploading csv files

In [40]:
df_Cleveland = pd.read_csv("/content/drive/MyDrive/CVD_DATA/cleveland.data", delimiter=" ")
df_Hungarian = pd.read_csv("/content/drive/MyDrive/CVD_DATA/hungarian.data", delimiter=" ")
df_Switzerland = pd.read_csv("/content/drive/MyDrive/CVD_DATA/switzerland.data", delimiter=" ")
df_LongBeachVA = pd.read_csv("/content/drive/MyDrive/CVD_DATA/long-beach-va.data", delimiter=" ")

In [41]:
df = pd.concat([df_Cleveland, df_Hungarian, df_Switzerland, df_LongBeachVA])

In [42]:
df_co = df[['smoke'
,'cigs'
,'years'
,'age'      
,'sex'       
,'cp'        
,'trestbps'
,'chol'   
,'fbs'       
,'restecg'
,'thalach'   
,'exang'    
,'oldpeak'
,'slope'    
,'ca'      
,'thal'
,'num']]

In [43]:
conditions = [
    (df_co['smoke'] == -9) & (df_co['cigs'] == -9) & (df_co['years'] == -9),
    (df_co['smoke'] == 0) | (df_co['cigs'] == 0) | (df_co['years'] == 0),
    (df_co['smoke'] > 0) | (df_co['cigs'] > 0) | (df_co['years'] > 0)
]

values = [-9, 0, 1]

df_co['is_smoker'] = np.select(conditions, values)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [45]:
df_smoker = df_co[['is_smoker', 'num']]

In [47]:
df_smoker = df_smoker[df_smoker['is_smoker'] != -9]

In [49]:
#########################################################
# Cross tabulation between is_smoker and num
CrosstabResult=pd.crosstab(index=df_smoker['is_smoker'],columns=df_smoker['num'])
print(CrosstabResult)
 
# importing the required function
from scipy.stats import chi2_contingency
 
# Performing Chi-sq test
ChiSqResult = chi2_contingency(CrosstabResult)
 
# P-Value is the Probability of H0 being True
# If P-Value&gt;0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSq Test is:', ChiSqResult[1])

num          0   1   2   3   4
is_smoker                     
0          106  47  32  37   4
1          109  70  46  42  19
The P-Value of the ChiSq Test is: 0.0358894065295935
